
# Youtube - Scaper



In [1]:
from pyppeteer import launch
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import time

# for extended documentation visit --> https://miyakogi.github.io/pyppeteer/
# !!! function could only be called with await !!!
async def scrape(url_: str, selector_: str, page_function_ = "(element) => element.outerHTML",
                 bypass_google_anti_scrape_algorithm_ = False, log_ = True):
    if log_ : print("-------------------------Scrape Log Begin--------------------------", "\n")
    #create random user agent so YouTube's algorithm gets pypassed
    ua = UserAgent()
    agent = ua.random
    
    # create browser, incognito context and page
    browser = await launch()
    context = await browser.createIncognitoBrowserContext()
    page = await context.newPage()
    if log_ : print("Browser, Incognito Context and Page created")
    
    # set user agent
    await page.setUserAgent(agent)
    if log_ : print("User Agent:", agent)
    
    # open url
    await page.goto(url_)
    if log_ : print("Url opened:", url_)
        
    if bypass_google_anti_scrape_algorithm_:
        await page.waitForSelector("h1.title")
        await page.click("h1.title")
        
        # multiple scroll to page end to get more comments
        for i in range(12):
            time.sleep(5)
            await page.keyboard.press("End")
    
    # wait until page gets loaded
    await page.waitForSelector(selector_)
    if log_ : print("Selector loaded:", selector_)
        
    await page.click(selector_)
        
    if bypass_google_anti_scrape_algorithm_:
        time.sleep(3)
        
    await page.click(selector_)
    
    # get element from query selector and relating function
    request_result = await page.querySelectorEval(selector_, page_function_)
    if log_ : print("Request finished")

    # close browser
    await browser.close()
    if log_ : print("Browser closed", "\n")
    if log_ : print("-------------------------Scrape Log End----------------------------", "\n")
    
    return request_result

## Let's test the "scrape" method: ##

In [2]:
# get YouTube Video Title

url = "https://www.youtube.com/watch?v=dyN_WtjdfpA&list=PLhTjy8cBISEoOtB5_nwykvB9wfEDscuEo"
query_selector = "h1.title"
function = "(element) => element.firstChild.innerHTML"

#title = await scrape(url, query_selector, function)                    
#print(title)

## Let's play around with BeautifulSoup for html parsing: ##

In [3]:
# get comments and their authors as html

url = "https://www.youtube.com/watch?v=dyN_WtjdfpA&list=PLhTjy8cBISEoOtB5_nwykvB9wfEDscuEo"
query_selector = "ytd-comments"
function = "(element) => element.outerHTML"

#html = await scrape(url, query_selector, function, True)

In [4]:
# parse html and assign them

def _parse_comments_with_corresponding_authors(html_, log_ = True):
    soup = BeautifulSoup(html_, features="html.parser")

    # get authors of comments and clear html data
    authors = [item.text.strip() for item in soup.select("a[id=author-text] > span")]

    # get comments and clear html data
    comments = [
        item.text.strip().replace("\r\n", " ").replace("\n", " ").replace("\"", "'") 
        for item in soup.select("yt-formatted-string[id=content-text]")
    ]
    #print(comments)

    likes = [
        item.text.strip().replace("\r\n", " ").replace("\n", " ").replace(".","").replace(",","")
        for item in soup.select("span[id=vote-count-middle]")
    ]
    
    #<span id="vote-count-middle" class="style-scope ytd-comment-action-buttons-renderer" aria-label="2&nbsp;&quot;Mag ich&quot;-Bewertungen">
    #print(likes)
    comments_with_authors_and_likes = list(zip(authors, comments, likes))

    
    if log_:
        print("Finished parsing")
        #for author, comment, likes in comments_with_authors_and_likes:
        #    print(author, "wrote:\n -" + comment + " with "+likes+" likes")
    
    return comments_with_authors_and_likes

# Let's try it:
#_parse_comments_with_corresponding_authors(html, False)

## Let's build parser methods: ##

In [5]:
# scrape and parse video metadata

# returns metadata as dict
# function is asynchronous and therefore it has to be awaited
async def _scrape_and_parse_video_meta_data(url: str, log_ = True):
    if "youtube.com" in url:
        
        trials = 1
        max_trials = 5
        
        while trials < max_trials:
            try:
                html = await scrape(
                    url, 
                    "div#info-contents",
                    "(element) => element.outerHTML", 
                    bypass_google_anti_scrape_algorithm_ = (trials > 2),
                    log_ = log_
                )
                
                trials = 100000
            except Exception as e:
                print('WARNING! : Metadata Scraping trial',trials,'failed for url="',url,'"!')
                print('ERROR:\n',e)
                trials += 1
                
                
        if trials == max_trials : 
            raise Exception("Meta-Data scraping trials all failed!! :(")
       
        
        soup = BeautifulSoup(html, features="html.parser")

        title = soup.find("h1", {"class": "title"}).find("yt-formatted-string").text
        primary_info = soup.find_all("yt-formatted-string", {"class": "ytd-video-primary-info-renderer"})
        
        date = (primary_info[len(primary_info) - 1].text)
    
        hashtags = [ tag.text.strip() for tag in primary_info[0].find_all("a") if tag != None]
        
        likes = soup.select("yt-formatted-string[id=text]")[0].text.replace(".", "").replace(",","").replace("\xa0Mio","0"*6)
        dislikes = soup.select("yt-formatted-string[id=text]")[1].text.replace(".", "").replace(",","").replace("\xa0Mio","0"*6)
        
        return {"title": title, "date": date, "hashtags": hashtags, "likes": likes, "dislikes": dislikes}
    else:
        print("Wrong url format given!")

In [6]:
# scrape and parse comments with authors

# returns list of tuples [(Author, Comment), (...), ...]
# function is asynchronous and therefore it has to be awaited
async def _scrape_and_parse_youtube_comments(url: str, log_ = True):
    
    if "youtube.com" in url:
        trials = 1
        max_trials = 5
        
        while trials < max_trials:
            try:
                html = await scrape(url, "ytd-comments", "(element) => element.outerHTML", bypass_google_anti_scrape_algorithm_ = True, log_ = log_)
        
                return _parse_comments_with_corresponding_authors(html, log_ = log_)
        
            except Exception as e:
                print('WARNING! : Comment scraping trial',trials,'failed for url="',url,'"!')
                print('ERROR:\n',e)
                trials += 1


        if trials == max_trials : 
            raise Exception("Comment scraping trials all failed!! :(")
    else:
        print("Wrong url format given!")
            

# Let's test it:      
#await _scrape_and_parse_youtube_comments("https://www.youtube.com/watch?v=dyN_WtjdfpA&list=PLhTjy8cBISEoOtB5_nwykvB9wfEDscuEo", log_ = False)

In [7]:
# Scraping transaction : 

def _scrape_and_store_video_metadata(tx, httpUrl_, metadata_): # "tx" is a neo4j transaction...
    #q = Pypher()
    merges = '\n'.join([
        "MERGE(t"+str(i)+':Tag{name:"'+str(t)+'"})\n' + 
        "MERGE(t"+str(i)+")-[:REFERENCES]->(v)" 
        for i, t in enumerate(metadata_['hashtags'])
    ]) 
    #q.CREATE.node('v', labels='Video')
    #q.SET.
    #for i, t in enumerate(metadata['hashtags']) :
    #    q.MERGE.node("t"+str(i))
        
    result = tx.run(
        "CREATE (v:Video) "
        "SET v = {title: $title, date: $date, likes: $likes, dislikes:$dislikes, url: $url}\n"+merges+"\n"
        "RETURN v.title + ', from node ' + id(v)", 
        title=metadata_["title"], 
        date=metadata_["date"], 
        likes=metadata_["likes"], 
        dislikes=metadata_["dislikes"],
        url=httpUrl_
    )
    print('Video Metadata"', metadata_,'" sent to database...')
    return result

def _scrape_and_store_video_comments(tx, httpUrl_, data):
    comments_with_authors_, ratio = data
    author_result = []
    comment_result = []
    for author, comment, likes in comments_with_authors_:
        author_result.append(tx.run(
            "MERGE (a:Author{name: $name})"
            "RETURN a.name + ', created as Author with id ' + id(a)", 
            name=author
        ))
        #print("Current comment:\n",comment)
        comment_result.append(tx.run(
            """
                MATCH (v:Video), (a:Author)
                WHERE v.url = "%s" AND a.name = "%s"
                CREATE (a) - [r:%s { text: "%s", likes: %s, score: %s }] -> (v)
                RETURN v.title, type(r), r.text, a.name
            """ % ( httpUrl_, author, "COMMENTED", comment, likes, ratio*int(likes))))
        
    print('Comments sent to database...')
    return zip(author_result, comment_result)

## Using Neo4j for data storage : ##

In [8]:
from neo4j import GraphDatabase

uri, user, password = 'bolt://localhost:7687', 'neo4j', 'neo4j_'


In [ ]:
httpUrls = [
    "https://www.youtube.com/watch?v=N_7eYLpynOs",
    "https://www.youtube.com/watch?v=tpiyEe_CqB4",
    "https://www.youtube.com/watch?v=HbaIfdEAkII",
    "https://www.youtube.com/watch?v=VoUD5MDfCEM",
    "https://www.youtube.com/watch?v=_pLTNTyD6nw",
    "https://www.youtube.com/watch?v=CAyWN9ba9J8",
    "https://www.youtube.com/watch?v=N274EurzpAA",
    "https://www.youtube.com/watch?v=L0H6xYwMQnk",
    "https://www.youtube.com/watch?v=rB83DpBJQsE",
    "https://www.youtube.com/watch?v=_DPRt3AcUEY",
    "https://www.youtube.com/watch?v=Sw9r8CL98N0",
    "https://www.youtube.com/watch?v=dyN_WtjdfpA",
    "https://www.youtube.com/watch?v=Ul0ZgDoamco",
    "https://www.youtube.com/watch?v=lcgqP8g6i84",
    "https://www.youtube.com/watch?v=KrEhmADXTr8",
    "https://www.youtube.com/watch?v=Iot0eF6EoNA",
    "https://www.youtube.com/watch?v=wx9Jv5uxfac",
    "https://www.youtube.com/watch?v=cV_uKzKZsHs",
    "https://www.youtube.com/watch?v=7plsgTxJn88",
    "https://www.youtube.com/watch?v=t-h1BhO4V_U",
    "https://www.youtube.com/watch?v=yufg9PdxrWs",
    "https://www.youtube.com/watch?v=tQ_nVSxjn_s",
    "https://www.youtube.com/watch?v=HtYweBOCp7A"
    "https://www.youtube.com/watch?v=L2c9q6zVp-I"
    "https://www.youtube.com/watch?v=hp_6u_MjjJ0",
    "https://www.youtube.com/watch?v=i9pNYW1Pg9A",
    "https://www.youtube.com/watch?v=MAlSjtxy5ak",
    "https://www.youtube.com/watch?v=pKO9UjSeLew",
    "https://www.youtube.com/watch?v=OTfp2_SwxHk",
    "https://www.youtube.com/watch?v=SzJ46YA_RaA",
    "https://www.youtube.com/watch?v=OmJ-4B-mS-Y",
    "https://www.youtube.com/watch?v=2a-CVGbUUjQ",
    "https://www.youtube.com/watch?v=cfDAdI2r-fw"
    "https://www.youtube.com/watch?v=MXJ-zpJeY3E",
    "https://www.youtube.com/watch?v=uTxRF5ag27A",
    "https://www.youtube.com/watch?v=IT__Nrr3PNI",
    "https://www.youtube.com/watch?v=C1vW9iSpLLk",
    "https://www.youtube.com/watch?v=kTXTPe3wahc",
    "https://www.youtube.com/watch?v=YbJOTdZBX1g",
    "https://www.youtube.com/watch?v=XUyoJ79P2SQ",
    "https://www.youtube.com/watch?v=CZIt20emgLY",
    "https://www.youtube.com/watch?v=XmIIgE7eSak",
    "https://www.youtube.com/watch?v=nIR2GR254S8",
    "https://www.youtube.com/watch?v=MO7erSNZszI",
    "https://www.youtube.com/watch?v=ASFSXNQKPDI",
    "https://www.youtube.com/watch?v=qjJcomKo_O8",
    "https://www.youtube.com/watch?v=ec65fZXcWv4",
    "https://www.youtube.com/watch?v=gFEjCuae8RA",
    "https://www.youtube.com/watch?v=i4XvlM_j3A0",
    "https://www.youtube.com/watch?v=1zZZjaYl4AA",
    "https://www.youtube.com/watch?v=qJW0LUE1RUA"
    "https://www.youtube.com/watch?v=LKOl8LoXPSY",
    "https://www.youtube.com/watch?v=RZEb_utxH4s",
    "https://www.youtube.com/watch?v=1vWTJzJx0i4",
    "https://www.youtube.com/watch?v=DxZVfwudvC8",
    "https://www.youtube.com/watch?v=BiqDZlAZygU",
    "https://www.youtube.com/watch?v=ulsQpeEUxx0",
    "https://www.youtube.com/watch?v=Iyt8rYpUQsg",
    "https://www.youtube.com/watch?v=3o3imqMkOYs",
    "https://www.youtube.com/watch?v=u1V-7UiXNZg",
    "https://www.youtube.com/watch?v=FJ3oHpup-pk",
    "https://www.youtube.com/watch?v=vgxCnXdHi68",
    "https://www.youtube.com/watch?v=QQ9gs-5lRKc",
    "https://www.youtube.com/watch?v=0jspaMLxBig",
    "https://www.youtube.com/watch?v=OiPVgAhRE6E",
    "https://www.youtube.com/watch?v=ggyVXHU0_ms",
    "https://www.youtube.com/watch?v=X6p5AZp7r_Q",
    "https://www.youtube.com/watch?v=DqgDqFrBnlQ",
    "https://www.youtube.com/watch?v=w3ugHP-yZXw",
    "https://www.youtube.com/watch?v=2lAe1cqCOXo",
    "https://www.youtube.com/watch?v=M0O7lLe4SmA",
    "https://www.youtube.com/watch?v=1RwQlgS4zQI",    
    "https://www.youtube.com/watch?v=ulsQpeEUxx0",
    "https://www.youtube.com/watch?v=3zhCbaVBEjg",
    "https://www.youtube.com/watch?v=067wXeqPCQU",
    "https://www.youtube.com/watch?v=QwZT7T-TXT0",
    "https://www.youtube.com/watch?v=rRAc29qoywA",
    "https://www.youtube.com/watch?v=o0vD7vqP1M8",
    "https://www.youtube.com/watch?v=FlsCjmMhFmw",
    "https://www.youtube.com/watch?v=2lAe1cqCOXo",
    "https://www.youtube.com/watch?v=vfc42Pb5RA8",
    "https://www.youtube.com/watch?v=NoQil9EWjxo",
    "https://www.youtube.com/watch?v=AFCGjW6Bwjs",
    "https://www.youtube.com/watch?v=WFcGOHEAypM",
    "https://www.youtube.com/watch?v=75U0YdxIi8M",
    "https://www.youtube.com/watch?v=cbqft9HbmDE",
    "https://www.youtube.com/watch?v=Iot0eF6EoNA",
    "https://www.youtube.com/watch?v=IvxRsDpXPGo",
    "https://www.youtube.com/watch?v=kffacxfA7G4",
    "https://www.youtube.com/watch?v=YbJOTdZBX1g",
    "https://www.youtube.com/watch?v=4gSOMba1UdM",
    "https://www.youtube.com/watch?v=EeF3UTkCoxY",
    "https://www.youtube.com/watch?v=kfVsfOSbJY0",
    "https://www.youtube.com/watch?v=hSlb1ezRqfA"
]

import random 
random.Random(66642999).shuffle(httpUrls)

httpUrls = list(set(httpUrls)) # removing duplicate videos if some exist...

driver = GraphDatabase.driver(uri, auth=(user, password))

# resetting database

with driver.session() as session:
    def _q(query) : return session.run(query)
    #---------------------------------------

    _q("MATCH (n) DETACH DELETE n") # remove all graphs and nodes! BE CAREFUL!

    #---------------------------------------
driver.close()

with driver.session() as session:
    for url in httpUrls :
        # run await outside of transaction because asynchronous transactions for Neo4j are not yet available for Python
        print("\n======================================================================================")
        print("| SCRAPING VIDEO : "+url)
        print("======================================================================================")
        try:
            metadata = await _scrape_and_parse_video_meta_data(url, log_ = True)
            result = session.write_transaction(_scrape_and_store_video_metadata, url, metadata)
            #print(result)

            comments_with_authors = await _scrape_and_parse_youtube_comments(url, log_ = True)
            if len(comments_with_authors) == 0 :
                print("Video without comments found! This might be wrong!")
                print("Let's try again...")
                comments_with_authors = await _scrape_and_parse_youtube_comments(url, log_ = True)

            ratio = (int(metadata['likes']))/(int(metadata['likes'])+int(metadata['dislikes']))*2 - 1 

            data = (comments_with_authors, ratio)
            result = session.write_transaction(_scrape_and_store_video_comments, url, data)
            #print(result)
        except :
            print('Failed loading url:', url)

driver.close()



| SCRAPING VIDEO : https://www.youtube.com/watch?v=BiqDZlAZygU
-------------------------Scrape Log Begin-------------------------- 

Browser, Incognito Context and Page created
User Agent: Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.3319.102 Safari/537.36
Url opened: https://www.youtube.com/watch?v=BiqDZlAZygU
Selector loaded: div#info-contents
Request finished
Browser closed 

-------------------------Scrape Log End---------------------------- 

Video Metadata" {'title': 'In full: Rowan Atkinson on free speech', 'date': '15.08.2018', 'hashtags': [], 'likes': '151029', 'dislikes': '1869'} " sent to database...
-------------------------Scrape Log Begin-------------------------- 

Browser, Incognito Context and Page created
User Agent: Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-TW) AppleWebKit/533.19.4 (KHTML, like Gecko) Version/5.0.2 Safari/533.18.5
Url opened: https://www.youtube.com/watch?v=BiqDZlAZygU
Selector loaded: ytd-comments
Request finish